In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install wget

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Download the library from the following link:
# https://artint.info/AIPython/

from os.path import exists
from pathlib import Path
from typing import Final
from zipfile import ZipFile

from wget import download

URL: Final[str] = 'https://artint.info/AIPython/aipython.zip'
DESTINATION: Final[Path] = Path('aipython.zip')

if not exists(DESTINATION):
    download(url=URL, out=DESTINATION.absolute().as_posix())

if not exists(DESTINATION.stem):
    with ZipFile(DESTINATION, 'r') as zip_ref:
        zip_ref.extractall()

In [3]:
import sys

# Cheat so I don't have to make `aipython` a package with setuptools ....
sys.path.append(DESTINATION.stem)

In [4]:
#;;
#;; Air cargo transport problem.
#;; http://www.inf.unibz.it/~tessaris/teaching/AI_06-07/labs/2007-01-10/planning.html
#;;

#(define (domain air-cargo)
#  (:requirements :strips)
#  (:predicates (In ?obj ?place)
#           (At ?obj ?place)
#           (Cargo ?obj)
#           (Plane ?obj)
#           (Airport ?obj))

#  (:action LOAD
#     :parameters (?c ?p ?a)
#     :precondition (and (At ?c ?a) (At ?p ?a)
#             (Cargo ?c) (Plane ?p) (Airport ?a))
#     :effect (and (In ?c ?p) (not (At ?c ?a))))
#  
#  (:action UNLOAD
#     :parameters (?c ?p ?a)
#     :precondition (and (In ?c ?p) (At ?p ?a)
#             (Cargo ?c) (Plane ?p) (Airport ?a))
#     :effect (and (At ?c ?a) (not (In ?c ?p))))
#  
#  (:action FLY
#     :parameters (?p ?from ?to)
#     :precondition (and (At ?p ?from)
#             (Plane ?p) (Airport ?from) (Airport ?to))
#     :effect (and (At ?p ?to) (not (At ?p ?from))))
#  )

In [5]:
# STRIPS domain - Air Cargo Transportation
from stripsProblem import STRIPS_domain, Strips

def In(obj):
    return obj+'_is_in'

def At(obj):
    return obj+"_is_at"

def Cargo(obj):
    return obj+"_is_cargo"

def Plane(obj):
    return obj+"_is_plane"

def Airport(obj):
    return obj+"_is_airport"

def Load(cargo, plane, airport):
    return "load_"+cargo+"_from_"+airport+"_to_"+plane

def Unload(cargo, plane, airport):
    return "unload_"+cargo+"_from_"+plane+"_to_"+airport

def Fly(plane, airport_from, airport_to):
    return "fly_"+plane+"_from_"+airport_from+"_to_"+airport_to


boolean = {True, False}
null = {'null'}
def create_air_cargo_transportation(cargo={'c1','c2'}, planes={'p1','p2'}, airports={'sfo', 'jfk'}):
    #feature_domain_dict = {Cargo(x):True for x in cargo}
    #feature_domain_dict.update({Cargo(x):False for x in cargo_storage})
    #feature_domain_dict.update({Plane(x):True for x in planes})
    #feature_domain_dict.update({Plane(x):False for x in cargo|airports})
    #feature_domain_dict.update({Airport(x):True for x in airports})
    #feature_domain_dict.update({Airport(x):False for x in cargo|planes})
    feature_domain_dict = {In(x):planes|null for x in cargo}
    feature_domain_dict.update({At(x):airports|null for x in planes|cargo})


    stmap = {Strips(Load(c, p, a), {At(c):a, At(p):a}, {In(c):p, At(c):'null'})
             for c in cargo
             for p in planes
             for a in airports}
    stmap.update({Strips(Unload(c, p, a), {In(c):p, At(p):a}, {At(c):a, In(c):'null'})
                  for c in cargo
                  for p in planes
                  for a in airports})

    stmap.update({Strips(Fly(p, a_from, a_to), {At(p):a_from}, {At(p):a_to})
                  for p in planes
                  for a_from in airports
                  for a_to in airports
                  if a_from != a_to})

    return STRIPS_domain(feature_domain_dict, stmap)







In [6]:
from stripsProblem import Planning_problem, STRIPS_domain
# Problem 1

air_cargo1dom = create_air_cargo_transportation()

air_setup1 = {
    At('c1'):'sfo',
    At('c2'):'jfk',
    At('p1'):'sfo',
    At('p2'):'jfk',
    In('c1'):'null',
    In('c2'):'null'}


air_cargo1 = Planning_problem(air_cargo1dom,
                              air_setup1, #initial state
                              {At('c1'):'jfk', At('c2'):'sfo', In('c1'):'null', In('c2'):'null'}) #goal


In [7]:
def h1(*args, **kwargs) -> int:
    return 0

In [8]:
#test
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS
SearcherMPP(Forward_STRIPS(air_cargo1, h1)).search()

Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null'}
   --load_c1_from_sfo_to_p1--> {'c1_is_at': 'null', 'c2_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null'}
   --fly_p1_from_sfo_to_jfk--> {'c1_is_at': 'null', 'c2_is_at': 'jfk', 'p1_is_at': 'jfk', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null'}
   --unload_c1_from_p1_to_jfk--> {'c1_is_at': 'jfk', 'c2_is_at': 'jfk', 'p1_is_at': 'jfk', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null'}
   --load_c2_from_jfk_to_p1--> {'c1_is_at': 'jfk', 'c2_is_at': 'null', 'p1_is_at': 'jfk', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p1'}
   --fly_p1_from_jfk_to_sfo--> {'c1_is_at': 'jfk', 'c2_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p1'}
   --unload_c2_from_p1_to_sfo--> {'c1_is_at': 'jfk', 'c2_is_at': 'sfo', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_i

{'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null'}
   --load_c1_from_sfo_to_p1--> {'c1_is_at': 'null', 'c2_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null'}
   --fly_p1_from_sfo_to_jfk--> {'c1_is_at': 'null', 'c2_is_at': 'jfk', 'p1_is_at': 'jfk', 'p2_is_at': 'jfk', 'c1_is_in': 'p1', 'c2_is_in': 'null'}
   --unload_c1_from_p1_to_jfk--> {'c1_is_at': 'jfk', 'c2_is_at': 'jfk', 'p1_is_at': 'jfk', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null'}
   --load_c2_from_jfk_to_p1--> {'c1_is_at': 'jfk', 'c2_is_at': 'null', 'p1_is_at': 'jfk', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p1'}
   --fly_p1_from_jfk_to_sfo--> {'c1_is_at': 'jfk', 'c2_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p1'}
   --unload_c2_from_p1_to_sfo--> {'c1_is_at': 'jfk', 'c2_is_at': 'sfo', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'nu

In [9]:
def h2(*args, **kwargs) -> int:
    return 0

In [10]:
# Problem 2
import stripsProblem

SearcherMPP(Forward_STRIPS(stripsProblem.problem1)).search()

Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_mr--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --puc--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --mc_cs--> {'RLoc': 'off', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --dc--> {'RLoc': 'off', 'MW': True, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 5)
 15 paths have been expanded and 11 paths remain in the frontier


{'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_mr--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --puc--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --mc_cs--> {'RLoc': 'off', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --dc--> {'RLoc': 'off', 'MW': True, 'SWC': False, 'RHC': False, 'RHM': False}

In [11]:
# Problem 3